In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score 
from sklearn.model_selection import learning_curve, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_train=pd.read_csv('train_qWM28Yl.csv')
df_test=pd.read_csv('test_zo1G9sv.csv')
ss=pd.read_csv('sample_submission_KvRh9Sx.csv')

In [4]:
df_train.head()

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0


In [5]:
df_train.describe()

,policy_tenure,age_of_car,age_of_policyholder,population_density,make,airbags,displacement,cylinder,gear_box,turning_radius,length,width,height,gross_weight,ncap_rating,is_claim
count,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.000000,58592.00000,58592.000000,58592.000000,58592.000000
mean,0.611246,0.069424,0.469420,18826.858667,1.763722,3.137066,1162.355851,3.626963,5.245443,4.852893,3850.476891,1672.233667,1553.33537,1385.276813,1.759950,0.063968
std,0.414156,0.056721,0.122886,17660.174792,1.136988,1.832641,266.304786,0.483616,0.430353,0.228061,311.457119,112.089135,79.62227,212.423085,1.389576,0.244698
min,0.002735,0.000000,0.288462,290.000000,1.000000,1.000000,796.000000,3.000000,5.000000,4.500000,3445.000000,1475.000000,1475.00000,1051.000000,0.000000,0.000000
25%,0.210250,0.020000,0.365385,6112.000000,1.000000,2.000000,796.000000,3.000000,5.000000,4.600000,3445.000000,1515.000000,1475.00000,1185.000000,0.000000,0.000000
50%,0.573792,0.060000,0.451923,8794.000000,1.000000,2.000000,1197.000000,4.000000,5.000000,4.800000,3845.000000,1735.000000,1530.00000,1335.000000,2.000000,0.000000
75%,1.039104,0.110000,0.548077,27003.000000,3.000000,6.000000,1493.000000,4.000000,5.000000,5.000000,3995.000000,1755.000000,1635.00000,1510.000000,3.000000,0.000000
max,1.396641,1.000000,1.000000,73430.000000,5.000000,6.000000,1498.000000,4.000000,6.000000,5.200000,4300.000000,1811.000000,1825.00000,1720.000000,5.000000,1.000000


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58592 entries, 0 to 58591
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   policy_id                         58592 non-null  object 
 1   policy_tenure                     58592 non-null  float64
 2   age_of_car                        58592 non-null  float64
 3   age_of_policyholder               58592 non-null  float64
 4   area_cluster                      58592 non-null  object 
 5   population_density                58592 non-null  int64  
 6   make                              58592 non-null  int64  
 7   segment                           58592 non-null  object 
 8   model                             58592 non-null  object 
 9   fuel_type                         58592 non-null  object 
 10  max_torque                        58592 non-null  object 
 11  max_power                         58592 non-null  object 
 12  engi

In [7]:
# No nulls in the df
print('training data dimensions', df_train.shape)
print('testing data dimensions', df_test.shape)

training data dimensions (58592, 44)
testing data dimensions (39063, 43)


## Training data preprocessing

In [8]:
# Listing all boolean value columns
bool_vars= [x for x in df_train.columns if df_train[x].values[0]=='Yes' or df_train[x].values[0]=='No']
bool_vars

['is_esc',
 'is_adjustable_steering',
 'is_tpms',
 'is_parking_sensors',
 'is_parking_camera',
 'is_front_fog_lights',
 'is_rear_window_wiper',
 'is_rear_window_washer',
 'is_rear_window_defogger',
 'is_brake_assist',
 'is_power_door_locks',
 'is_central_locking',
 'is_power_steering',
 'is_driver_seat_height_adjustable',
 'is_day_night_rear_view_mirror',
 'is_ecw',
 'is_speed_alert']

In [9]:
# Now convering them to yes=1, no=0
def binary_map(x):
    return x.map({'Yes': 1, "No": 0})

df_train[bool_vars]= df_train[bool_vars].apply(binary_map)
df_train.head()

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,0,0,0,1,0,0,0,1,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,0,0,0,1,0,0,0,1,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,0,0,0,1,0,0,0,1,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,1,1,1,1,1,1,1,1,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,0,1,1,1,0,1,1,1,2,0


In [10]:
df_train.drop('policy_id', axis=1, inplace=True)
df_train.make=df_train.make.astype(str)

In [11]:
cat_cols= [x for x in df_train.columns if df_train[x].dtype=='O']
cat_cols

['area_cluster',
 'make',
 'segment',
 'model',
 'fuel_type',
 'max_torque',
 'max_power',
 'engine_type',
 'rear_brakes_type',
 'transmission_type',
 'steering_type']

In [12]:
dummy1= pd.get_dummies(df_train[cat_cols], drop_first=True)
dummy1

,area_cluster_C10,area_cluster_C11,area_cluster_C12,area_cluster_C13,area_cluster_C14,area_cluster_C15,area_cluster_C16,area_cluster_C17,area_cluster_C18,area_cluster_C19,...,engine_type_1.5 Turbocharged Revotron,engine_type_F8D Petrol Engine,engine_type_G12B,engine_type_K Series Dual jet,engine_type_K10C,engine_type_i-DTEC,rear_brakes_type_Drum,transmission_type_Manual,steering_type_Manual,steering_type_Power
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
58588,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,1
58589,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,1
58590,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,0,0


In [13]:
df_train.shape

(58592, 43)

In [14]:
df_train = pd.concat([df_train, dummy1], axis=1) 
df_train.drop(cat_cols,axis=1, inplace=True)

Need to resample the data becuase the data is highly imbalanced as only 6% data shows the claiments 

In [15]:
from sklearn.utils import resample
#create two different dataframe of majority and minority class 
df_majority = df_train[(df_train['is_claim']==0)] 
df_minority = df_train[(df_train['is_claim']==1)] 
# upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,    # sample with replacement
                                 n_samples= 54844, # to match majority class
                                 random_state=42)  # reproducible results
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority])


## Train test split

In [16]:
y= df_upsampled.pop('is_claim')
X= df_upsampled.copy()

In [17]:
# possibility of a claim 
sum(y)/len(y)*100

50.0

In [18]:
# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

### Model Building

In [41]:
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import f1_score

xgb1 = xgb.XGBClassifier(
 learning_rate =0.01,
 n_estimators=1000,
 max_depth=10,
 min_child_weight=1,
 gamma=0,
 subsample=0.6,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

xgb1.fit(X_train, y_train)
predicts = xgb1.predict(X_test)
print(accuracy_score(y_test, predicts))
print(f1_score(y_test, predicts))

0.8567781930896162
0.8696590060565834


In [20]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)
score_rf = round(accuracy_score(y_pred_rf,y_test)*100,2)
print("The accuracy score we have achieved using Decision Tree is: "+str(score_rf)+" %")

The accuracy score we have achieved using Decision Tree is: 97.99 %


In [21]:
print(f1_score(y_test, y_pred_rf))

0.9803524648725888


In [22]:
print('Training Accuracy : ',
      accuracy_score(y_train,
                             rf.predict(X_train))*100)
print('Validation Accuracy : ',
      accuracy_score(y_test,
                             rf.predict(X_test))*100)

Training Accuracy :  100.0
Validation Accuracy :  97.99434770717477


In [23]:
# Now we got the final model we need to apply the predictions on testing data 
df_test.head()

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
0,ID58593,0.341732,0.00,0.586538,C3,4076,1,A,M1,CNG,...,No,No,No,No,Yes,No,No,No,Yes,0
1,ID58594,0.307241,0.13,0.442308,C8,8794,1,B2,M6,Petrol,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2
2,ID58595,0.327924,0.12,0.451923,C8,8794,2,A,M3,Petrol,...,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2
3,ID58596,0.782654,0.01,0.461538,C5,34738,1,A,M1,CNG,...,No,No,No,No,Yes,No,No,No,Yes,0
4,ID58597,1.233404,0.02,0.634615,C5,34738,1,A,M1,CNG,...,No,No,No,No,Yes,No,No,No,Yes,0


In [24]:
bool_vars_test= [x for x in df_test.columns if df_test[x].values[0]=='Yes' or df_test[x].values[0]=='No']
bool_vars_test

['is_esc',
 'is_adjustable_steering',
 'is_tpms',
 'is_parking_sensors',
 'is_parking_camera',
 'is_front_fog_lights',
 'is_rear_window_wiper',
 'is_rear_window_washer',
 'is_rear_window_defogger',
 'is_brake_assist',
 'is_power_door_locks',
 'is_central_locking',
 'is_power_steering',
 'is_driver_seat_height_adjustable',
 'is_day_night_rear_view_mirror',
 'is_ecw',
 'is_speed_alert']

In [25]:
df_test[bool_vars_test]= df_test[bool_vars_test].apply(binary_map)
df_test.head()

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
0,ID58593,0.341732,0.00,0.586538,C3,4076,1,A,M1,CNG,...,0,0,0,0,1,0,0,0,1,0
1,ID58594,0.307241,0.13,0.442308,C8,8794,1,B2,M6,Petrol,...,0,1,1,1,1,1,1,1,1,2
2,ID58595,0.327924,0.12,0.451923,C8,8794,2,A,M3,Petrol,...,0,0,1,1,1,0,1,1,1,2
3,ID58596,0.782654,0.01,0.461538,C5,34738,1,A,M1,CNG,...,0,0,0,0,1,0,0,0,1,0
4,ID58597,1.233404,0.02,0.634615,C5,34738,1,A,M1,CNG,...,0,0,0,0,1,0,0,0,1,0


In [26]:
df_test.drop('policy_id', axis=1, inplace=True)
df_test.make=df_test.make.astype(str)
cat_cols_test= [x for x in df_test.columns if df_test[x].dtype=='O']
cat_cols_test

['area_cluster',
 'make',
 'segment',
 'model',
 'fuel_type',
 'max_torque',
 'max_power',
 'engine_type',
 'rear_brakes_type',
 'transmission_type',
 'steering_type']

In [27]:
dummy1_test= pd.get_dummies(df_test[cat_cols_test], drop_first=True)
dummy1_test

,area_cluster_C10,area_cluster_C11,area_cluster_C12,area_cluster_C13,area_cluster_C14,area_cluster_C15,area_cluster_C16,area_cluster_C17,area_cluster_C18,area_cluster_C19,...,engine_type_1.5 Turbocharged Revotron,engine_type_F8D Petrol Engine,engine_type_G12B,engine_type_K Series Dual jet,engine_type_K10C,engine_type_i-DTEC,rear_brakes_type_Drum,transmission_type_Manual,steering_type_Manual,steering_type_Power
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39058,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,0,0
39059,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
39060,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
39061,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [28]:
df_test = pd.concat([df_test, dummy1_test], axis=1) 
df_test.drop(cat_cols_test,axis=1, inplace=True)
print(df_test.shape)
print(df_test.head())

(39063, 103)
   policy_tenure  age_of_car  age_of_policyholder  population_density  \
0       0.341732        0.00             0.586538                4076   
1       0.307241        0.13             0.442308                8794   
2       0.327924        0.12             0.451923                8794   
3       0.782654        0.01             0.461538               34738   
4       1.233404        0.02             0.634615               34738   

   airbags  is_esc  is_adjustable_steering  is_tpms  is_parking_sensors  \
0        2       0                       0        0                   1   
1        2       0                       1        0                   1   
2        2       0                       0        0                   0   
3        2       0                       0        0                   1   
4        2       0                       0        0                   1   

   is_parking_camera  ...  engine_type_1.5 Turbocharged Revotron  \
0                  0  ...    

In [29]:
predictions_final= rf.predict(df_test)


In [30]:
ss.head()

,policy_id,is_claim
0,ID58593,1
1,ID58594,1
2,ID58595,1
3,ID58596,1
4,ID58597,1


In [31]:
df_test_1=pd.read_csv('test_zo1G9sv.csv')

In [32]:
df_final= pd.DataFrame({'policy_id':df_test_1.policy_id, 'is_claim':predictions_final})
df_final.reset_index(drop=True, inplace=True)
df_final.head()

,policy_id,is_claim
0,ID58593,0
1,ID58594,0
2,ID58595,0
3,ID58596,0
4,ID58597,1


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
# Turn the values into an array for feeding the classification algorithms.
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [35]:
# Let's implement simple classifiers

classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier()
}

In [36]:
# Wow our scores are getting even high scores even when applying cross validation.
'''from sklearn.model_selection import cross_val_score


for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    training_score = cross_val_score(classifier, X_train, y_train, cv=5)
    print("Classifiers: ", key, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")
'''

'from sklearn.model_selection import cross_val_score\n\n\nfor key, classifier in classifiers.items():\n    classifier.fit(X_train, y_train)\n    training_score = cross_val_score(classifier, X_train, y_train, cv=5)\n    print("Classifiers: ", key, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")\n'

In [61]:
for i in range(1,6):
    knn= KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    y_preds_1 = knn.predict(X_test)
    scores = round(accuracy_score(y_preds_1,y_test)*100,2)
    print("The accuracy score we have achieved using KNN is: "+str(scores)+" %")
    print(f1_score(y_preds_1,y_test))

The accuracy score we have achieved using KNN is: 96.3 %
0.9646218194492855
The accuracy score we have achieved using KNN is: 96.3 %
0.9646218194492855
The accuracy score we have achieved using KNN is: 92.69 %
0.932564889992007
The accuracy score we have achieved using KNN is: 92.69 %
0.9325086257679038
The accuracy score we have achieved using KNN is: 89.45 %
0.9054214160232055


In [58]:
knn= KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
y_preds_1 = knn.predict(X_test)
scores = round(accuracy_score(y_preds_1,y_test)*100,2)
print("The accuracy score we have achieved using KNN is: "+str(scores)+" %")
print(f1_score(y_preds_1,y_test))

The accuracy score we have achieved using KNN is: 92.69 %
0.9325086257679038


In [59]:
df_final= pd.DataFrame({'policy_id':df_test_1.policy_id, 'is_claim':knn.predict(df_test)})
df_final.reset_index(drop=True, inplace=True)
df_final.head()

,policy_id,is_claim
0,ID58593,1
1,ID58594,0
2,ID58595,0
3,ID58596,0
4,ID58597,1


In [60]:
# Final submission
df_final.to_csv('final_submission.csv')